<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Diabetes prediction using BYOM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>According to research, there are a lot of people that are suffering from diabetes all over the world. Studies show that in 2019, diabetes was the direct cause of 1.5 million deaths and almost 50% of all deaths occurred before the age of 70.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  
Over time, diabetes can have a negative impact on multiple organs. It can damage the heart, blood vessels, eyes, kidneys, and nerves. The earlier a person receives a proper treatment, the more likely he or she will be in lowering blood glucose level. In addition, the risk of failure of other organs will also be reduced.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  
Hence as a data science consultant, we are showcasing the complete approach about how we can make prediction of diagnosis of  diabetes 6 months in advance. We are demonstrating how we can bring our models that were trained with open-source technologies to Teradata Vantage for scoring. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Comprehensive health predictions and a reduced number of false positive and false negative results.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Reduced cost to patients and hospitals caused by heart attacks.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Identify patterns and symptoms leading to heart failure to ensure early intervention.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Advanced research and development stemming from the results of the data and models produced.</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ML and AI industry continues to innovate at an unprecedented rate. Tools, technologies, and algorithms are being developed and improved in both the open source and commercial communities.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Unfortunately, many of these techniques haven’t matured to the point where they are readily deployable to a stable, mature operational environment. Furthermore, many open-source techniques rely on fragile, manual enabling technologies. Traditional approaches require the developers to move data from the sources to the analytics. Even "integrated" analytic systems like Apache Spark provide parallel processing for analyzing data but don't optimize for loading data - neither locality nor quantity that needs to be moved. Teradata Vantage reverses this model and allows PUSH processing down to the individual processing nodes where the data resides. This allows for the unprecedented scale of the analytical processing, reduced costs in data movement/egress charges, and drastically improved performance.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A critical strategy for Vantage and ClearScape Analytics is to embrace the value and innovation in the open-source and partner ML and AI community. A cornerstone of that strategy is to allow users to leverage their ML or AI tools and models of choice to deploy those models directly to the Vantage Platform.  This provides enterprises with the most scalable option for deploying custom machine learning pipelines. Users can leverage the innovation and familiarity of a broad range of tools and techniques, with the ability to prepare and score new data in near-real-time and at any scale; allowing the products of machine learning to become pervasive across all applications, reporting tools, and consumers in an organization.
</p>
 

 



<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
# Import necessary libraries.
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import json
import getpass
import pandas as pd
from teradataml import *

import teradatasql
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn import metrics

# import teradataml
# from teradataml.dataframe.dataframe import in_schema
# from teradataml.catalog.byom import save_byom, retrieve_byom
# from teradataml.analytics.byom import H2OPredict

# import sqlalchemy
# from sqlalchemy import event
# from sqlalchemy.types import String
# from teradataml.context.context import *
# from teradataml.dataframe.dataframe import DataFrame
# from sqlalchemy.types import VARCHAR
# from teradataml.dataframe.copy_to import copy_to_sql
display.max_rows=5

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Diabetes_Classification_BYOM.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage.  Here we are downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. We are only creating local databases and tables as there are 625 columns in table which will be faster in local tables as compared to foreign tables.</p> 
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>“Note:  The data loading part of this demo will be slow because we have large number of  columns (625 columns)."</b></p>  


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_DiabetesPrediction_BYOM_local');"
 # Takes about 4 minutes 


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us start by creating a "Virtual DataFrame" that points directly to the dataset in Vantage. We then begin our analysis by checking the shape of the DataFrame and examining the data types of all its columns.</p>




In [ ]:
df_byom=DataFrame(in_schema("DEMO_DiabetesPrediction_BYOM","Diabetes_Diagnosis_BYOM_Data"))
df_byom

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Bringing our Trained H2O Model to Vantage

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/BYOM/save_byom'>save_byom</a> function allows users to save various models stored in different formats such as PMML, MOJO, and so on.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>Model_ID specifies the unique model identifier for this model</li>
        <li>Model_File specifies the absolute path of the file which has model information</li>
    <li>Other parameter values detailed in the documentation</li>
        </ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have our trained model in the form of zip file. In order to use it in-database, we need to save it to the table.</p>    

In [ ]:
query = """
CREATE SET TABLE DBT_H2O_Models (
  model_id VARCHAR (30),
  model BLOB
)
PRIMARY INDEX (model_id);"""

try:
    execute_sql(query)
except:
    db_drop_table('DBT_H2O_Models')
    execute_sql(query)

In [ ]:
save_byom(model_id="dbt_model_1", model_file="GBM_2_AutoML_1_20230214_131128.zip", table_name="DBT_H2O_Models")

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Model Scoring
 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-Package-for-Python-User-Guide-17.20/BYOM/retrieve_byom'>retrieve_byom</a> API allows a user to retrieve a saved model. Output of this function can be directly passed as input to the PMMLPredict and H2OPredict functions.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>Model_ID specifies the unique model identifier of the model to be retrieved</li>
    <li>Other parameter values detailed in the documentation</li>
        </ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After bringing our model to Vantage, we need to score it on the data available in Vantage to check its performance.</p> 

In [ ]:
df_data = DataFrame(in_schema("DEMO_DiabetesPrediction_BYOM", "Diabetes_Diagnosis_BYOM_Data"))

In [ ]:
modeldata = retrieve_byom("dbt_model_1", table_name="DBT_H2O_Models")

configure.byom_install_location = "mldb"

result = H2OPredict(newdata=df_data,
                    newdata_partition_column='MBR_ID',
                    newdata_order_column='MBR_ID',
                    modeldata=modeldata,
                    modeldata_order_column='model_id',
                    model_output_fields=['classProbabilities'],
                    # model_output_fields=['prob_0','prob_1'],
                    accumulate=['MBR_ID'],
                    overwrite_cached_models='*',
                    enable_options=['contributions','stageProbabilities'],
                    model_type='OpenSource'
                    )
df_predict = result.result

In [ ]:
Target = df_data.select(['MBR_ID', 'target'])
# DataFrame.from_query("SELECT MBR_ID, target FROM DEMO_DiabetesPrediction_BYOM.Diabetes_Diagnosis_BYOM_Data;")
Result_df = df_predict.merge(Target,how='inner', on="MBR_ID", lsuffix = 't1', rsuffix='t2')
Result_df = Result_df.assign(drop_columns=True, 
                             MBR_ID = Result_df.t1_MBR_ID,
                             target = Result_df.target,
                             prediction = Result_df.prediction,
                             classprobabilities = Result_df.classprobabilities)
Result_df  

In [ ]:
Result=Result_df.to_pandas()

In [ ]:
def Predict(x):
    if x["1"] >= 0.5:
        return 1
    return 0
def Prob(x):
    return x["1"]
# Target = df_data.select(['MBR_ID', 'target'])
# # DataFrame.from_query("SELECT MBR_ID, target FROM DEMO_DiabetesPrediction_BYOM.Diabetes_Diagnosis_BYOM_Data;")
# Result = df_predict.merge(Target.to_pandas(),how='inner', on="MBR_ID")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We are using the class probabilities returned by H2OPredict to classify the prediction as 0 or 1.</p>

In [ ]:
Result["classprobabilities"].isna().count()

In [ ]:
Result["classprobabilities"].index

In [ ]:
Result["classprobabilities"].apply(json.loads)

In [ ]:
Result["classprobabilities"] = Result["classprobabilities"].apply(json.loads)

In [ ]:
Result["Prediction"] = Result["classprobabilities"].apply(Predict)

In [ ]:
Result["Prob_1"] = Result["classprobabilities"].apply(Prob)

In [ ]:
Result

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Create CONFUSION MATRIX.</p>

In [ ]:
cm = confusion_matrix(Result['target'], Result['Prediction'])
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['DoesNotHaveDiabetes', 'HasDiabetes'])
cmd.plot()
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above Confusion Matrix shows the actual and the Predicted values. Based on H2O model used the above matrix shows the predicted and actual value comparison for people having Diabetes and those not having Diabetes.</p>


In [ ]:
Result_df = Result_df.assign(drop_columns=True, 
                             MBR_ID = Result_df.MBR_ID,
                             target = Result_df.target.cast(type_=INTEGER),
                             prediction = Result_df.prediction.cast(type_=INTEGER))                           

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(data=Result_df,
                                                          observation_column='target',
                                                          prediction_column='prediction',
                                                          labels=['0', '1'])
classeval_df = ClassificationEvaluator_obj.output_data
classeval_df

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above output has the macro and weighted-average metrics of precision, recall, and F1-score values.</p>
<table style = 'font-size:16px;font-family:Arial;color:#00233C'>
  <tr>
    <th>Column</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Precision</td>
    <td>The positive predictive value. Refers to the fraction of relevant instances among
the total retrieved instances.
        Precision answers the following question: what proportion of predicted Positives is truly Positive? 
        Precision = (TP)/(TP+FP)</td>
  </tr>
  <tr>
    <td>Recall</td>
    <td>Refers to the fraction of relevant instances retrieved over the total amount of
relevant instances. Recall answers a different question: what proportion of actual Positives is correctly classified?
Recall = (TP)/(TP+FN)</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>F1 score, defined as the harmonic mean of the precision and recall and is a number between 0 and 1. F1 score maintains a balance between the precision and recall for your classifier.                                         
                      F1 = 2*(precision*recall/precision+recall)</td>
  </tr>
  <tr>
    <td>Support</td>
    <td>The number of times a label displays in the Observation Column.</td>
  </tr>
</table>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>**TP:- True Positive , FP :- False Positive, TN :- True Negative , FN :- False Negative

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Show AUC-ROC Curve</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create AUC-ROC curve with the sk-learn roc_curve() Function.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ROC curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the TPR(true-positive rate) against FPR(false-positive rate).The ROC curve is plotted with TPR against the FPR where TPR is on the y-axis and FPR is on the x-axis.</p>

In [ ]:
Result= Result.drop('Prediction', axis=1)
Result = Result.drop('classprobabilities', axis=1)

In [ ]:
copy_to_sql(Result,table_name='result_table',if_exists='replace')
df_Result =  DataFrame('result_table')

In [ ]:
roc_df = ROC(data = df_Result, 
                    probability_column = "Prob_1",
                    observation_column = "target",
                    positive_class="1"
                    )
roc_df.output_data

In [ ]:
auc = roc_df.result.get_values()[0][0]
auc

In [ ]:
plot_roc_df = roc_df.output_data
plot =  plot_roc_df.plot(x=plot_roc_df.fpr, y=plot_roc_df.tpr,
                         title="Receiver Operating Characteristic (ROC) Curve",
                         xlabel='False Positive Rate', 
                         ylabel='True Positive Rate', 
                         color="blue",
                         legend=f'AUC = {round(auc, 4)}',
                         legend_style='lower right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)
 
# Display the plot.
plot.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus, with Bring Your Own Model capabilities of ClearScape Analytics we allow organizations to leverage third party and open-source models for scoring inside the Vantage Platform, providing enterprise-class scalability and operational stability for any number of users, applications, or volume of data.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>7. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>

In [ ]:
db_drop_table('DBT_H2O_Models')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_DiabetesPrediction_BYOM');" 
#Takes 2 minutes

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Resources </b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b> Data</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This dataset contains data for 10000 patients, half of which were diagnosed with diabetes. It contains 624 columns. We used patients' visit records consisting of diagnoses, procedures, medications and demographics. In addition, we also added a temporal aspect to the medical features. We differentiated between events occurring 1-3 months before diagnosis, 3-6 months, and 6-12 months, before the prediction window. The main aim of the data is to distinguish between healthy people and those who were diagnosed with diabetes, according to "target" column which is set to 0 for non-diabetic and 1 for diabetic.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://synthea.mitre.org/'>Link to the dataset</a>: This dataset was generated by Synthea for the experimentation purpose and does not reflect the actual population.</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Healthcare</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Machine Learning</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Prediction Analysis</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Blogs/Forecasting-COVID-19-Using-Teradata-Vantage'>Forecasting COVID-19 Using Teradata Vantage</a>
    </li></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>